In [1]:
import requests
import json
url = "https://animepahe.ru/"
search_url_extension = "api?m=search&q="

keyword = "Boku no hero"

full_search_url = url+search_url_extension+keyword


response = requests.get(full_search_url)
results = json.loads(response.content.decode("UTF-8"))["data"]

[{'id': 877, 'title': 'Boku no Hero Academia', 'type': 'TV', 'episodes': 13, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2016, 'score': 7.9, 'poster': 'https://i.animepahe.com/posters/bf74acb64682024d7b49626fa69fe822aa403a2a7690c2c739a291833c8bae3c.jpg', 'session': '00ee7744-bb3b-f0f4-fcfc-9677114c4240'}, {'id': 1719, 'title': 'Boku no Hero Academia 2nd Season', 'type': 'TV', 'episodes': 25, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2017, 'score': 8.12, 'poster': 'https://i.animepahe.com/posters/4a8396f9f5ddcae0b2f42b672139d697c243857a3bf55ac3f21ac920fda01d09.jpg', 'session': '958f0580-3f58-3009-deaf-16aa79210e5a'}, {'id': 2477, 'title': 'Boku no Hero Academia 3rd Season', 'type': 'TV', 'episodes': 25, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2018, 'score': 8.05, 'poster': 'https://i.animepahe.com/posters/606327773e492b2528e1d12ad07951d607a7d93ee384bc5ab2dc8d591308dd17.jpg', 'session': '24fd1d80-68ee-3f4b-a37f-9a4dd69e1bb8'}, {'id': 3429, 'ti